In [1]:
import buttermilk

# Configuration files are stored in the local directory, and
# options can be passed in at initialization.
# For notebooks we might need to initialize separately:
# (this will be fixed later, I just can't figure out the path/cwd problem yet)
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
import hydra
from omegaconf import OmegaConf

abs_config_dir=os.getcwd() + "/../conf"
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    cfg = compose(config_name="config", overrides=["+flows=simple", "llm=full"])

objs = hydra.utils.instantiate(cfg)
bm = objs.bm
logger = bm.logger
logger.info("Starting interactive run for climate frames in notebook")

# print config details
from rich import print as rprint
rprint("\nConfiguration:")
rprint(OmegaConf.to_yaml(cfg))


/opt/conda/envs/python311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Project(
    name='buttermilk',
    job='debugging',
    connections=[],
    secret_provider=CloudProviderCfg(
        type='gcp',
        project='prosocial-443205',
        models_secret='dev__llm__connections'
    ),
    save_dest=CloudProviderCfg(
        type='gcp',
        project='prosocial-443205',
        region='us-central1',
        bucket='prosocial-dev'
    ),
    logger=CloudProviderCfg(type='gcp'),
    pubsub=CloudProviderCfg(type='gcp', project='prosocial-443205', subscription='flow-sub'),
    clouds=[
        CloudProviderCfg(type='vertex', project='prosocial-443205', region='us-central1', bucket='prosocial-dev')
    ],
    flows=[],
    tracing=Tracing(enabled=True, endpoint=None, otlp_headers={}),
    verbose=True,
    run=RunCfg(platform='local', max_concurrency=32, parameters={}, flow_api='http://flow:8000/flow/')
)

SessionInfo(
    run_id='20241219T0624Z-WeYQ-d04c5ab214fc-debian',
    ip='159.196.210.27',
    node_name='d04c5ab214fc',
    username='debian',
    save_dir='/tmp/tmp3_q5m9hx'
)

Successfully dumped to json on disk: /tmp/tmp3_q5m9hx/tmp4w_w1jev.jsonl.


Prompt flow service has started...


Configuration:

bm:
  _target_: buttermilk.BM
  cfg:
    name: buttermilk
    job: debugging
    save_dest:
      type: gcp
      project: prosocial-443205
      region: us-central1
      bucket: prosocial-dev
    clouds:
    - type: vertex
      project: prosocial-443205
      region: us-central1
      bucket: prosocial-dev
    logger:
      type: gcp
    verbose: true
    tracing:
      enabled: true
    run:
      platform: local
      max_concurrency: 32
      flow_api: http://flow:8000/flow/
    secret_provider:
      type: gcp
      project: prosocial-443205
      models_secret: dev__llm__connections
    pubsub:
      type: gcp
      project: prosocial-443205
      subscription: flow-sub
save:
  type: bq
  dataset: dmrc-analysis.toxicity.flow
  db_schema: flow.json
llm:
- gpt4o
- llama32_90b
- sonnet
- gemini15pro
flows:
  simple:
    _target_: buttermilk.runner.flow.Flow
    source: hate development
    steps:
    - name: describer
      _target_: buttermilk.agents.describer.Describer
      parameters:
        template: describe
        model:
        - gemini15pro
      inputs:
        record: record
      outputs:
        record: record
    - _target_: buttermilk.agents.lc.LC
      name: judger
      num_runs: 1
      concurrency: 20
      save: ${save}
      parameters:
        template: judge
        criteria: criteria_ordinary
        formatting: json_rules
        model: ${llm}
      inputs:
        record: record
      outputs:
        job_id: job_id
        model: parameters.model
        criteria: parameters.criteria
        template: parameters.template
        reasons: outputs.reasons

## Send a simple request by creating a new record

In [2]:
import shortuuid

from buttermilk._core.runner_types import RecordInfo

results = []
record = RecordInfo(text="kill all men")

# Load the flow from the config
flow = objs.flows['simple']

# Run through each of the steps in this flow with this new record
async for data in flow.run_flows(
    record=record,
    run_info=bm._run_metadata,
    source="notebook",
    flow_id=shortuuid.uuid(),
    q=None
):
    rprint(data.outputs)
    results.append(data)


ERROR:buttermilk:Response data not formatted as expected: 'NoneType' object has no attribute 'model_dump', e.args=("'NoneType' object has no attribute 'model_dump'",)


None

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=buttermilk&uiTraceId=0xee85e7a150274bb6aa3ee5e7e8e449b9
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=buttermilk&uiTraceId=0xbad3557ad9dfdb72afb99265b3212784
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=buttermilk&uiTraceId=0xccfc41961ce95da7f5cea76abab1875c
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=buttermilk&uiTraceId=0xcdf0aa48f830e1c59d8988d42078da5a
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=buttermilk&uiTraceId=0x4c1ef5fdf93eca86389ba077f4fd1441


ERROR:buttermilk:Error invoking chain with gpt4o after 1.72 seconds: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}} 
ERROR:buttermilk:Agent judger failed with error: {'judger': "Error invoking chain with gpt4o after 1.72 seconds: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}} "}


Result(
    category=None,
    prediction=None,
    result=None,
    labels=[],
    confidence=None,
    severity=None,
    reasons=[],
    scores=None,
    metadata=None
)

domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1beta1.PredictionService.GenerateContent"
}
].
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1beta1.PredictionService.GenerateContent"
}
].


Result(
    category=None,
    prediction=False,
    result=None,
    labels=[],
    confidence='high',
    severity='N/A',
    reasons=[
        'The phrase "kill all men" is a violent statement that expresses a desire for the death of all males. While
shocking, the literal meaning of the phrase is clear. The phrase has been used online and is associated with 
particular strains of feminist thought, some of which are more extreme than others. In some contexts, the phrase 
has been used ironically or satirically to express anger or frustration with men or patriarchal structures. In 
other contexts, the phrase is used more literally but often hyperbolically, as an expression of rage or despair. In
still other contexts, the phrase has been adopted by extremist groups.  Determining toxicity depends on applying 
the criteria below to make an assessment about how the phrase would likely be interpreted in context, given the 
speaker and the audience.',
        'RULE 1, TARGETS A MARGINALIZED GROUP: This content targets men, who are not a marginalized group. Although
individual men can be subject to discrimination, and subgroups of men might be marginalized, men as a whole are not
subject to systemic discrimination.',
        'RULE 2, PRESENCE OF A POWER IMBALANCE:  It is difficult to assess the relative power of the speaker 
without more context. Given the wide dissemination of the phrase "kill all men", it\'s likely that many different 
people, speaking from different positions and levels of social power, will use the phrase. However, given that it 
targets men as a general group and men are not systemically marginalized, it is unlikely that the phrase is coming 
from a position of relative powerlessness in a way that could be seen as speaking back to power.',
        'RULE 3, SUBORDINATION: This content expresses a desire for violence against men and treats them as 
deserving of death. This could be interpreted as subordinating if it was found to otherwise target a marginalized 
group.'
    ],
    scores=None,
    metadata=None
)

Result(
    category=None,
    prediction=False,
    result=None,
    labels=[],
    confidence='high',
    severity='N/A',
    reasons=[
        'The phrase "kill all men" appears to target men as a group, often used as a provocative feminist slogan or
expression of frustration with patriarchal systems. While inflammatory, this phrase emerged as a form of rhetorical
hyperbole in feminist discourse, typically used to critique systemic sexism rather than literally advocate 
violence.',
        'While the content targets men as a group, men as a category have not historically faced systemic 
discrimination. Though the language is hostile, it does not target a marginalized population protected under this 
policy.',
        "Though the speaker's identity is unknown, this type of rhetoric typically comes from those pushing back 
against patriarchal power structures rather than from a position of institutional power over men. The content 
appears to be speaking back to power rather than exercising power over a marginalized group.",
        'While the phrase is hostile and suggests harm, it does not operate to subordinate men as a group since men
continue to hold relative social, economic and political power. The rhetorical nature of the phrase means it 
functions more as critique than actual subordination.'
    ],
    scores=None,
    metadata=None
)

domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1beta1.PredictionService.GenerateContent"
}
].
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1beta1.PredictionService.GenerateContent"
}
].
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1beta1.PredictionService.GenerateContent"
}
].
ERROR:buttermilk:Error invoking chain with llama32_90b after 37.94 seconds: 400 Invalid resource field value in the request. [reason: "RESOURCE_PROJECT_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "method"
  value: "google.cloud.aiplatform.v1beta1.PredictionService.GenerateContent"
}
] 
ERROR:buttermilk:Agent judger faile

Result(
    category=None,
    prediction=None,
    result=None,
    labels=[],
    confidence=None,
    severity=None,
    reasons=[],
    scores=None,
    metadata=None
)